In [17]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import numpy as np
import random
import json
from pprint import pprint

In [4]:
def box_office_1000(url, year):
    col_names=['rank', 'title', 'worldwide $', 'domestic $', 'domestic %', 'foreign $', 'foreign %']
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    container = soup.find('div', class_='a-section imdb-scroll-table-inner')
    tables = container.find('table').findAll('tr')
    for f in range(1,101):
        time.sleep(random.uniform(1, 3))
        film_row = tables[f]
        cols = film_row.findAll('td')
        row_dict={}
        row_dict['year']=year
        c=0
        for col in cols:
            col_name = col_names[c]
            row_dict[col_name] = col.text
            c+=1
        box_office_list.append(row_dict)

        

In [5]:
box_office_list = []

def next_year():
    for d in range(10):
        year = 2021 - d
        string_year = str(year)
        base_url = 'https://www.boxofficemojo.com/year/world/'
        url = base_url + string_year + "/"
        box_office_1000(url, string_year)    
        
next_year()
box_office_1000_df = pd.DataFrame(box_office_list)
box_office_1000_df

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom","$822,009,764",-,-,"$822,009,764",100%
1,2021,2,F9: The Fast Saga,"$714,398,815","$172,931,815",24.2%,"$541,467,000",75.8%
2,2021,3,Detective Chinatown 3,"$686,257,563",-,-,"$686,257,563",100%
3,2021,4,Godzilla vs. Kong,"$467,863,133","$100,563,133",21.5%,"$367,300,000",78.5%
4,2021,5,Black Widow,"$377,665,240","$183,177,333",48.5%,"$194,487,907",51.5%
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,"$68,267,862","$35,353,000",51.8%,"$32,914,862",48.2%
996,2012,97,Moonrise Kingdom,"$68,263,166","$45,512,466",66.7%,"$22,750,700",33.3%
997,2012,98,The Secret of the Gods,"$67,537,798",-,-,"$67,537,798",100%
998,2012,99,Madea's Witness Protection,"$66,899,242","$65,653,242",98.1%,"$1,246,000",1.9%


In [6]:
box_office_1000_df.dtypes

year           object
rank           object
title          object
worldwide $    object
domestic $     object
domestic %     object
foreign $      object
foreign %      object
dtype: object

In [49]:
bo_1000_clean_df = box_office_1000_df.applymap(lambda x: x[1:].replace(',', '') if  x[0] == "$" else x)
bo_1000_clean_df = bo_1000_clean_df.applymap(lambda x: x[:-1] if  x[-1] == "%" else x)
bo_1000_clean_df

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom",822009764,-,-,822009764,100
1,2021,2,F9: The Fast Saga,714398815,172931815,24.2,541467000,75.8
2,2021,3,Detective Chinatown 3,686257563,-,-,686257563,100
3,2021,4,Godzilla vs. Kong,467863133,100563133,21.5,367300000,78.5
4,2021,5,Black Widow,377665240,183177333,48.5,194487907,51.5
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,68267862,35353000,51.8,32914862,48.2
996,2012,97,Moonrise Kingdom,68263166,45512466,66.7,22750700,33.3
997,2012,98,The Secret of the Gods,67537798,-,-,67537798,100
998,2012,99,Madea's Witness Protection,66899242,65653242,98.1,1246000,1.9


In [50]:
bo_1000_clean_df = bo_1000_clean_df.applymap(lambda x: '0' if  x == "-" else x)
bo_1000_clean_df.head(1000)

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom",822009764,0,0,822009764,100
1,2021,2,F9: The Fast Saga,714398815,172931815,24.2,541467000,75.8
2,2021,3,Detective Chinatown 3,686257563,0,0,686257563,100
3,2021,4,Godzilla vs. Kong,467863133,100563133,21.5,367300000,78.5
4,2021,5,Black Widow,377665240,183177333,48.5,194487907,51.5
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,68267862,35353000,51.8,32914862,48.2
996,2012,97,Moonrise Kingdom,68263166,45512466,66.7,22750700,33.3
997,2012,98,The Secret of the Gods,67537798,0,0,67537798,100
998,2012,99,Madea's Witness Protection,66899242,65653242,98.1,1246000,1.9


In [54]:
bo_1000_clean_df[['domestic %', 'foreign %']] = bo_1000_clean_df[['domestic %', 'foreign %']].applymap(lambda x: x[1:] if x[0] == '<' else x)
bo_1000_clean_df.head(1000)

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom",822009764,0,0,822009764,100
1,2021,2,F9: The Fast Saga,714398815,172931815,24.2,541467000,75.8
2,2021,3,Detective Chinatown 3,686257563,0,0,686257563,100
3,2021,4,Godzilla vs. Kong,467863133,100563133,21.5,367300000,78.5
4,2021,5,Black Widow,377665240,183177333,48.5,194487907,51.5
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,68267862,35353000,51.8,32914862,48.2
996,2012,97,Moonrise Kingdom,68263166,45512466,66.7,22750700,33.3
997,2012,98,The Secret of the Gods,67537798,0,0,67537798,100
998,2012,99,Madea's Witness Protection,66899242,65653242,98.1,1246000,1.9


In [56]:

bo_1000_clean_df[['worldwide $', 'domestic $', 'foreign $', 'domestic %', 'foreign %']] = bo_1000_clean_df[['worldwide $', 'domestic $', 'foreign $', 'domestic %', 'foreign %']].astype(float)
bo_1000_clean_df

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom",822009764.0,0.0,0.0,822009764.0,100.0
1,2021,2,F9: The Fast Saga,714398815.0,172931815.0,24.2,541467000.0,75.8
2,2021,3,Detective Chinatown 3,686257563.0,0.0,0.0,686257563.0,100.0
3,2021,4,Godzilla vs. Kong,467863133.0,100563133.0,21.5,367300000.0,78.5
4,2021,5,Black Widow,377665240.0,183177333.0,48.5,194487907.0,51.5
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,68267862.0,35353000.0,51.8,32914862.0,48.2
996,2012,97,Moonrise Kingdom,68263166.0,45512466.0,66.7,22750700.0,33.3
997,2012,98,The Secret of the Gods,67537798.0,0.0,0.0,67537798.0,100.0
998,2012,99,Madea's Witness Protection,66899242.0,65653242.0,98.1,1246000.0,1.9


In [57]:

bo_1000_clean_df.head(1000)

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom",822009764.0,0.0,0.0,822009764.0,100.0
1,2021,2,F9: The Fast Saga,714398815.0,172931815.0,24.2,541467000.0,75.8
2,2021,3,Detective Chinatown 3,686257563.0,0.0,0.0,686257563.0,100.0
3,2021,4,Godzilla vs. Kong,467863133.0,100563133.0,21.5,367300000.0,78.5
4,2021,5,Black Widow,377665240.0,183177333.0,48.5,194487907.0,51.5
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,68267862.0,35353000.0,51.8,32914862.0,48.2
996,2012,97,Moonrise Kingdom,68263166.0,45512466.0,66.7,22750700.0,33.3
997,2012,98,The Secret of the Gods,67537798.0,0.0,0.0,67537798.0,100.0
998,2012,99,Madea's Witness Protection,66899242.0,65653242.0,98.1,1246000.0,1.9


In [58]:
bo_1000_clean_df.to_csv('top_1000_bo.csv')

In [3]:
df = pd.read_csv('top_1000_bo.csv')
df.head(1000)

,Unnamed: 0,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,0,2021,1,"Hi, Mom",822009764.0,0.0,0.0,822009764.0,100.0
1,1,2021,2,F9: The Fast Saga,714398815.0,172931815.0,24.2,541467000.0,75.8
2,2,2021,3,Detective Chinatown 3,686257563.0,0.0,0.0,686257563.0,100.0
3,3,2021,4,Godzilla vs. Kong,467863133.0,100563133.0,21.5,367300000.0,78.5
4,4,2021,5,Black Widow,377665240.0,183177333.0,48.5,194487907.0,51.5
...,...,...,...,...,...,...,...,...,...
995,995,2012,96,The Watch,68267862.0,35353000.0,51.8,32914862.0,48.2
996,996,2012,97,Moonrise Kingdom,68263166.0,45512466.0,66.7,22750700.0,33.3
997,997,2012,98,The Secret of the Gods,67537798.0,0.0,0.0,67537798.0,100.0
998,998,2012,99,Madea's Witness Protection,66899242.0,65653242.0,98.1,1246000.0,1.9


In [181]:
df = df[['title', 'year', 'worldwide $', 'domestic $', 'domestic %', 'foreign $', 'foreign %']]
df
df.to_csv('df.csv')

,title,year,worldwide $,domestic $,domestic %,foreign $,foreign %
0,"Hi, Mom",2021,822009764.0,0.0,0.0,822009764.0,100.0
1,F9: The Fast Saga,2021,714398815.0,172931815.0,24.2,541467000.0,75.8
2,Detective Chinatown 3,2021,686257563.0,0.0,0.0,686257563.0,100.0
3,Godzilla vs. Kong,2021,467863133.0,100563133.0,21.5,367300000.0,78.5
4,Black Widow,2021,377665240.0,183177333.0,48.5,194487907.0,51.5
...,...,...,...,...,...,...,...
995,The Watch,2012,68267862.0,35353000.0,51.8,32914862.0,48.2
996,Moonrise Kingdom,2012,68263166.0,45512466.0,66.7,22750700.0,33.3
997,The Secret of the Gods,2012,67537798.0,0.0,0.0,67537798.0,100.0
998,Madea's Witness Protection,2012,66899242.0,65653242.0,98.1,1246000.0,1.9


In [187]:
df.to_csv('df.csv')

In [182]:
english_df = df[df['domestic %'] >= 5]
non_english_df = df[df['domestic %'] <= 5]



In [188]:
english_df.to_csv('english_df.csv')
non_english_df.to_csv('non_english_df.csv')

In [184]:
english_df.insert(4, "imdb_code", "No_code")

In [186]:
english_df.set_index('imdb_code')

,title,year,worldwide $,domestic $,domestic %,foreign $,foreign %
imdb_code,,,,,,,
No_code,F9: The Fast Saga,2021,714398815.0,172931815.0,24.2,541467000.0,75.8
No_code,Godzilla vs. Kong,2021,467863133.0,100563133.0,21.5,367300000.0,78.5
No_code,Black Widow,2021,377665240.0,183177333.0,48.5,194487907.0,51.5
No_code,A Quiet Place Part II,2021,297372261.0,160072261.0,53.8,137300000.0,46.2
No_code,Free Guy,2021,277748887.0,102882486.0,37.0,174866401.0,63.0
...,...,...,...,...,...,...,...
No_code,Anna Karenina,2012,68929150.0,12816367.0,18.6,56112783.0,81.4
No_code,The Watch,2012,68267862.0,35353000.0,51.8,32914862.0,48.2
No_code,Moonrise Kingdom,2012,68263166.0,45512466.0,66.7,22750700.0,33.3


In [27]:
with open('C:/Users/james/.secret/patronOmdb.txt', 'r') as f:
    omdb_data = f.read()


In [190]:
english_df.iloc[0,0]

'F9: The Fast Saga'

In [164]:
def url_maker(year, title):
    url = 'http://www.omdbapi.com/?t=' + title + '&y=' + year + '&apikey=' + omdb_data
    return url

def title_corrector(title):
    if title == 'F9: The Fast Saga':
        title = 'F9'
    return title

def get_page(url):
    response=requests.get(url)
    movie_data = json.loads(response.content.decode('utf-8')) 
    return movie_data

dud_url=[]
retrieve_error=[]
no_error = []

for m in range(724):
    movie = english_df.iloc[m,0]
    title = movie.replace(',', '%2C').replace (' ', '+').replace (':', '%3A')
    year = str(df.iloc[m,1])
    url = url_maker(year, movie)
    try:
        movie_data = get_page(url)
        if "Error" in movie_data:  
            prev_year = (str(int(year) -1))
            url = url_maker(prev_year, movie)
            movie_data = get_page(url)
            if "Error" in movie_data:
                alt_title = title_corrector(movie)
                url = url_maker(api_year, alt_title)
                movie_data = get_page(url)
                if "Error" in movie_data:
                    film_error = movie + " - " + movie_data_t['Error'] + "\n"
                    retrieve_error.append(prev_film_error)
                else:
                    
                    # add idmb, download the relevant keys 
                    no_error.append(movie)  
            else:
                no_error.append(movie)  
        else:
            no_error.append(movie)
    except:
        dud_url.append(movie)
        
#print(dud_url)
#print(retrieve_error)
#print(no_error)

['Demon Slayer: Mugen Train', 'Dragon Ball Super: Broly', 'Baahubali 2: The Conclusion', 'The Commuter', 'The Angry Birds Movie', 'Teenage Mutant Ninja Turtles 2: Out of the Shadows', 'The Divergent Series: Insurgent', 'Ride Along 2', 'The 5th Wave', 'Crimson Peak', "Lee Daniels' The Butler", 'Jurassic Park 2013 Re-release', 'Instructions Not Included', 'Scary Movie V', 'Titanic 2012 3D Release', "We're the Millers", 'Last Vegas', 'Walking with Dinosaurs 3D', 'Star Wars: Episode I - The Phantom Menace 2012 3D Release', 'Finding Nemo 2012 3D Release', 'Beauty and the Beast 2012 3D Release']
[]
['The Suicide Squad', 'Space Jam: A New Legacy', 'Peter Rabbit 2: The Runaway', 'Tom and Jerry', 'Raya and the Last Dragon', 'The Boss Baby: Family Business', 'Wrath of Man', 'PAW Patrol: The Movie', 'Old', 'Mortal Kombat', 'The Forever Purge', "Hitman's Wife's Bodyguard", 'Candyman', 'Nobody', "Don't Breathe 2", 'Escape Room: Tournament of Champions', 'In the Heights', 'Spirit Untamed', 'Spiral',

In [18]:
testeri=requests.get('http://www.omdbapi.com/?i=tt0112641&apikey=727131f3')
jsoned=json.loads(testeri.content.decode('utf-8')) 
pprint(jsoned)

{'Actors': 'Robert De Niro, Sharon Stone, Joe Pesci',
 'Awards': 'Nominated for 1 Oscar. 4 wins & 11 nominations total',
 'BoxOffice': '$42,512,375',
 'Country': 'United States, France',
 'DVD': '26 Feb 2002',
 'Director': 'Martin Scorsese',
 'Genre': 'Crime, Drama',
 'Language': 'English',
 'Metascore': '73',
 'Plot': 'A tale of greed, deception, money, power, and murder occur between '
         'two best friends: a mafia enforcer and a casino executive compete '
         'against each other over a gambling empire, and over a fast-living '
         'and fast-loving socialite.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTcxOWYzNDYtYmM4YS00N2NkLTk0NTAtNjg1ODgwZjAxYzI3XkEyXkFqcGdeQXVyNTA4NzY1MzY@._V1_SX300.jpg',
 'Production': 'Universal Pictures',
 'Rated': 'R',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.2/10'},
             {'Source': 'Rotten Tomatoes', 'Value': '79%'},
             {'Source': 'Metacritic', 'Value': '73/100'}],
 'Released': '22 Nov 1995',

In [160]:
testert=requests.get('http://www.omdbapi.com/?t=Impasse&y=2021&apikey=727131f3')
print(testert.content)

b'{"Response":"False","Error":"Movie not found!"}'


In [8]:
eng_df = pd.read_csv('english_df.csv')
eng_df

,Unnamed: 0,title,year,worldwide $,domestic $,imdb_code,domestic %,foreign $,foreign %
0,1,F9: The Fast Saga,2021,714398815.0,172931815.0,No_code,24.2,541467000.0,75.8
1,3,Godzilla vs. Kong,2021,467863133.0,100563133.0,No_code,21.5,367300000.0,78.5
2,4,Black Widow,2021,377665240.0,183177333.0,No_code,48.5,194487907.0,51.5
3,5,A Quiet Place Part II,2021,297372261.0,160072261.0,No_code,53.8,137300000.0,46.2
4,6,Free Guy,2021,277748887.0,102882486.0,No_code,37.0,174866401.0,63.0
...,...,...,...,...,...,...,...,...,...
719,994,Anna Karenina,2012,68929150.0,12816367.0,No_code,18.6,56112783.0,81.4
720,995,The Watch,2012,68267862.0,35353000.0,No_code,51.8,32914862.0,48.2
721,996,Moonrise Kingdom,2012,68263166.0,45512466.0,No_code,66.7,22750700.0,33.3
722,998,Madea's Witness Protection,2012,66899242.0,65653242.0,No_code,98.1,1246000.0,1.9


In [7]:
eng_df.set_index('imdb_code')

,Unnamed: 0,title,year,worldwide $,domestic $,domestic %,foreign $,foreign %
imdb_code,,,,,,,,
No_code,1,F9: The Fast Saga,2021,714398815.0,172931815.0,24.2,541467000.0,75.8
No_code,3,Godzilla vs. Kong,2021,467863133.0,100563133.0,21.5,367300000.0,78.5
No_code,4,Black Widow,2021,377665240.0,183177333.0,48.5,194487907.0,51.5
No_code,5,A Quiet Place Part II,2021,297372261.0,160072261.0,53.8,137300000.0,46.2
No_code,6,Free Guy,2021,277748887.0,102882486.0,37.0,174866401.0,63.0
...,...,...,...,...,...,...,...,...
No_code,994,Anna Karenina,2012,68929150.0,12816367.0,18.6,56112783.0,81.4
No_code,995,The Watch,2012,68267862.0,35353000.0,51.8,32914862.0,48.2
No_code,996,Moonrise Kingdom,2012,68263166.0,45512466.0,66.7,22750700.0,33.3


In [9]:
eng_df = eng_df[['imdb_code', 'title', 'year', 'worldwide $', 'domestic $', 'domestic %', 'foreign $', 'foreign %']]
eng_df

,imdb_code,title,year,worldwide $,domestic $,domestic %,foreign $,foreign %
0,No_code,F9: The Fast Saga,2021,714398815.0,172931815.0,24.2,541467000.0,75.8
1,No_code,Godzilla vs. Kong,2021,467863133.0,100563133.0,21.5,367300000.0,78.5
2,No_code,Black Widow,2021,377665240.0,183177333.0,48.5,194487907.0,51.5
3,No_code,A Quiet Place Part II,2021,297372261.0,160072261.0,53.8,137300000.0,46.2
4,No_code,Free Guy,2021,277748887.0,102882486.0,37.0,174866401.0,63.0
...,...,...,...,...,...,...,...,...
719,No_code,Anna Karenina,2012,68929150.0,12816367.0,18.6,56112783.0,81.4
720,No_code,The Watch,2012,68267862.0,35353000.0,51.8,32914862.0,48.2
721,No_code,Moonrise Kingdom,2012,68263166.0,45512466.0,66.7,22750700.0,33.3
722,No_code,Madea's Witness Protection,2012,66899242.0,65653242.0,98.1,1246000.0,1.9


In [24]:
eng_df.iloc[2,0]

'No_code'

In [26]:
with open('C:/Users/james/.secret/imdbApi.txt', 'r') as g:
    imdb_data = g.read()
imdb_data

'k_12345678'

In [48]:
omdb_attrs = {}
dud_url=[]
retrieve_error=[]

def on_success(movie_data, eng_df, m):
    imdb_id = movie_data['imdbID']  
    eng_df.loc[eng_df.index[m], 'imdb_code'] = imdb_id
    keys = ['Actors', 'Director', 'Genre', 'Plot', 'Rated', 'Ratings', 'Runtime', 'Writer', 'Title', 'imdbRating', 'imdbVotes']
    omdb_attrs[imdb_id] = {x:movie_data[x] for x in keys}

def eng_url_maker(year, title):
    url = 'http://www.omdbapi.com/?t=' + title + '&y=' + year + '&apikey=' + omdb_data
    return url

def eng_title_corrector(title):
    if title == 'F9: The Fast Saga':
        title = 'F9'
    return title

def eng_get_page(url):
    response=requests.get(url)
    movie_data = json.loads(response.content.decode('utf-8')) 
    return movie_data


def make_omdb_table(eng_df):
    for m in range(723):
        movie = eng_df.iloc[m,1]
        title = movie.replace(',', '%2C').replace (' ', '+').replace (':', '%3A')
        year = str(eng_df.iloc[m,2])
        url = eng_url_maker(year, title)
        try:
            movie_data = eng_get_page(url)
            if "Error" in movie_data:  
                prev_year = (str(int(year) -1))
                url = eng_url_maker(prev_year, title)
                movie_data = eng_get_page(url)
                if "Error" in movie_data:
                    alt_title = eng_title_corrector(movie)
                    url = eng_url_maker(year, alt_title)
                    movie_data = eng_get_page(url)
                    if "Error" in movie_data:
                        film_error = movie + " - " + movie_data_t['Error'] + "\n"
                        retrieve_error.append(prev_film_error)
                    else:
                        on_success(movie_data, eng_df, m)
                else:
                    on_success(movie_data, eng_df, m)  
            else:
                on_success(movie_data, eng_df, m)
        except:
            dud_url.append(movie)

make_omdb_table(eng_df)


C:\Users\james\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [49]:
eng_df

,imdb_code,title,year,worldwide $,domestic $,domestic %,foreign $,foreign %
0,tt5433138,F9: The Fast Saga,2021,714398815.0,172931815.0,24.2,541467000.0,75.8
1,tt5034838,Godzilla vs. Kong,2021,467863133.0,100563133.0,21.5,367300000.0,78.5
2,tt3480822,Black Widow,2021,377665240.0,183177333.0,48.5,194487907.0,51.5
3,tt8332922,A Quiet Place Part II,2021,297372261.0,160072261.0,53.8,137300000.0,46.2
4,tt6264654,Free Guy,2021,277748887.0,102882486.0,37.0,174866401.0,63.0
...,...,...,...,...,...,...,...,...
719,tt1781769,Anna Karenina,2012,68929150.0,12816367.0,18.6,56112783.0,81.4
720,tt1298649,The Watch,2012,68267862.0,35353000.0,51.8,32914862.0,48.2
721,tt1748122,Moonrise Kingdom,2012,68263166.0,45512466.0,66.7,22750700.0,33.3
722,tt2215285,Madea's Witness Protection,2012,66899242.0,65653242.0,98.1,1246000.0,1.9


In [54]:
eng_df.set_index('imdb_code')
eng_df

,imdb_code,title,year,worldwide $,domestic $,domestic %,foreign $,foreign %
0,tt5433138,F9: The Fast Saga,2021,714398815.0,172931815.0,24.2,541467000.0,75.8
1,tt5034838,Godzilla vs. Kong,2021,467863133.0,100563133.0,21.5,367300000.0,78.5
2,tt3480822,Black Widow,2021,377665240.0,183177333.0,48.5,194487907.0,51.5
3,tt8332922,A Quiet Place Part II,2021,297372261.0,160072261.0,53.8,137300000.0,46.2
4,tt6264654,Free Guy,2021,277748887.0,102882486.0,37.0,174866401.0,63.0
...,...,...,...,...,...,...,...,...
719,tt1781769,Anna Karenina,2012,68929150.0,12816367.0,18.6,56112783.0,81.4
720,tt1298649,The Watch,2012,68267862.0,35353000.0,51.8,32914862.0,48.2
721,tt1748122,Moonrise Kingdom,2012,68263166.0,45512466.0,66.7,22750700.0,33.3
722,tt2215285,Madea's Witness Protection,2012,66899242.0,65653242.0,98.1,1246000.0,1.9


In [55]:
# eng_lang films at box office
eng_df.to_csv('eng_df.csv')

In [77]:
for key in omdb_attrs:
    for k in omdb_attrs[key]['Ratings']:
        outlet_ratings = list(k.values())
        if outlet_ratings[0] == 'Rotten Tomatoes':
            omdb_attrs[key]['RottenRating'] = outlet_ratings[1][:-1]
        elif outlet_ratings[0] == 'Metacritic':
            omdb_attrs[key]['MetacriticRating'] = outlet_ratings[1][:-4]
pprint(omdb_attrs)

#'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.2/10'},
         #    {'Source': 'Rotten Tomatoes', 'Value': '79%'},
          #   {'Source': 'Metacritic', 'Value': '73/100'}],

{'tt0293429': {'Actors': 'Lewis Tan, Jessica McNamee, Josh Lawson',
               'Director': 'Simon McQuoid',
               'Genre': 'Action, Adventure, Fantasy',
               'MetacriticRating': '44',
               'Plot': "MMA fighter Cole Young seeks out Earth's greatest "
                       'champions in order to stand against the enemies of '
                       'Outworld in a high stakes battle for the universe.',
               'Rated': 'R',
               'Ratings': [{'Source': 'Internet Movie Database',
                            'Value': '6.1/10'},
                           {'Source': 'Rotten Tomatoes', 'Value': '55%'},
                           {'Source': 'Metacritic', 'Value': '44/100'}],
               'RottenRating': '55',
               'Runtime': '110 min',
               'Title': 'Mortal Kombat',
               'Writer': 'Greg Russo, Dave Callaham, Oren Uziel',
               'imdbRating': '6.1',
               'imdbVotes': '133,256'},
 'tt0359950': {'A

                'RottenRating': '48',
                'Runtime': '90 min',
                'Title': 'Unhinged',
                'Writer': 'Carl Ellsworth',
                'imdbRating': '6.0',
                'imdbVotes': '55,843'},
 'tt1024648': {'Actors': 'Ben Affleck, Bryan Cranston, John Goodman',
               'Director': 'Ben Affleck',
               'Genre': 'Biography, Drama, Thriller',
               'MetacriticRating': '86',
               'Plot': 'Acting under the cover of a Hollywood producer '
                       'scouting a location for a science fiction film, a CIA '
                       'agent launches a dangerous operation to rescue six '
                       'Americans in Tehran during the U.S. hostage crisis in '
                       'Iran in 1979.',
               'Rated': 'R',
               'Ratings': [{'Source': 'Internet Movie Database',
                            'Value': '7.7/10'},
                           {'Source': 'Rotten Tomatoes', 'Value': '9

 'tt1211837': {'Actors': 'Benedict Cumberbatch, Chiwetel Ejiofor, Rachel '
                         'McAdams',
               'Director': 'Scott Derrickson',
               'Genre': 'Action, Adventure, Fantasy',
               'MetacriticRating': '72',
               'Plot': 'While on a journey of physical and spiritual healing, '
                       'a brilliant neurosurgeon is drawn into the world of '
                       'the mystic arts.',
               'Rated': 'PG-13',
               'Ratings': [{'Source': 'Internet Movie Database',
                            'Value': '7.5/10'},
                           {'Source': 'Rotten Tomatoes', 'Value': '89%'},
                           {'Source': 'Metacritic', 'Value': '72/100'}],
               'RottenRating': '89',
               'Runtime': '115 min',
               'Title': 'Doctor Strange',
               'Writer': 'Jon Spaihts, Scott Derrickson, C. Robert Cargill',
               'imdbRating': '7.5',
               'imdbVote

               'imdbRating': '7.7',
               'imdbVotes': '471,020'},
 'tt1408253': {'Actors': 'Ice Cube, Kevin Hart, Tika Sumpter',
               'Director': 'Tim Story',
               'Genre': 'Action, Comedy, Crime',
               'MetacriticRating': '41',
               'Plot': 'Security guard Ben must prove himself to his '
                       "girlfriend's brother, top police officer James. He "
                       'rides along James on a 24-hour patrol of Atlanta.',
               'Rated': 'PG-13',
               'Ratings': [{'Source': 'Internet Movie Database',
                            'Value': '6.2/10'},
                           {'Source': 'Rotten Tomatoes', 'Value': '18%'},
                           {'Source': 'Metacritic', 'Value': '41/100'}],
               'RottenRating': '18',
               'Runtime': '99 min',
               'Title': 'Ride Along',
               'Writer': 'Greg Coolidge, Jason Mantzoukas, Phil Hay',
               'imdbRating': '6.2

                       'Cobra; they are forced to contend with threats from '
                       'within the government that jeopardize their very '
                       'existence.',
               'Rated': 'PG-13',
               'Ratings': [{'Source': 'Internet Movie Database',
                            'Value': '5.8/10'},
                           {'Source': 'Rotten Tomatoes', 'Value': '29%'},
                           {'Source': 'Metacritic', 'Value': '41/100'}],
               'RottenRating': '29',
               'Runtime': '110 min',
               'Title': 'G.I. Joe: Retaliation',
               'Writer': 'Rhett Reese, Paul Wernick',
               'imdbRating': '5.8',
               'imdbVotes': '177,074'},
 'tt1586265': {'Actors': 'Cameron Diaz, Matthew Morrison, J. Todd Smith',
               'Director': 'Kirk Jones',
               'Genre': 'Comedy, Drama, Romance',
               'MetacriticRating': '41',
               'Plot': 'Follows the lives of five intercon

...

               'MetacriticRating': '67',
               'Plot': 'The Guardians struggle to keep together as a team '
                       'while dealing with their personal family issues, '
                       "notably Star-Lord's encounter with his father the "
                       'ambitious celestial being Ego.',
               'Rated': 'PG-13',
               'Ratings': [{'Source': 'Internet Movie Database',
                            'Value': '7.6/10'},
                           {'Source': 'Rotten Tomatoes', 'Value': '85%'},
                           {'Source': 'Metacritic', 'Value': '67/100'}],
               'RottenRating': '85',
               'Runtime': '136 min',
               'Title': 'Guardians of the Galaxy Vol. 2',
               'Writer': 'James Gunn, Dan Abnett, Andy Lanning',
               'imdbRating': '7.6',
               'imdbVotes': '608,202'},
 'tt4034228': {'Actors': 'Casey Affleck, Michelle Williams, Kyle Chandler',
               'Director': 'Kenne

               'Ratings': [{'Source': 'Internet Movie Database',
                            'Value': '7.7/10'},
                           {'Source': 'Rotten Tomatoes', 'Value': '97%'},
                           {'Source': 'Metacritic', 'Value': '86/100'}],
               'RottenRating': '97',
               'Runtime': '147 min',
               'Title': 'Mission: Impossible - Fallout',
               'Writer': 'Bruce Geller, Christopher McQuarrie',
               'imdbRating': '7.7',
               'imdbVotes': '305,997'},
 'tt4913966': {'Actors': 'Linda Cardellini, Raymond Cruz, Patricia Velasquez',
               'Director': 'Michael Chaves',
               'Genre': 'Horror, Mystery, Thriller',
               'MetacriticRating': '41',
               'Plot': 'Ignoring the eerie warning of a troubled mother '
                       'suspected of child endangerment, a social worker and '
                       'her own small kids are soon drawn into a frightening '
                   

               'Plot': 'Six years after the events of "Wreck-It Ralph," Ralph '
                       'and Vanellope, now friends, discover a wi-fi router in '
                       'their arcade, leading them into a new adventure.',
               'Rated': 'PG',
               'Ratings': [{'Source': 'Internet Movie Database',
                            'Value': '7.0/10'},
                           {'Source': 'Rotten Tomatoes', 'Value': '88%'},
                           {'Source': 'Metacritic', 'Value': '71/100'}],
               'RottenRating': '88',
               'Runtime': '112 min',
               'Title': 'Ralph Breaks the Internet',
               'Writer': 'Phil Johnston, Pamela Ribon, Rich Moore',
               'imdbRating': '7.0',
               'imdbVotes': '142,693'},
 'tt5884052': {'Actors': 'Ryan Reynolds, Justice Smith, Kathryn Newton',
               'Director': 'Rob Letterman',
               'Genre': 'Action, Adventure, Comedy',
               'MetacriticRating'

                            'Value': '5.6/10'},
                           {'Source': 'Rotten Tomatoes', 'Value': '38%'},
                           {'Source': 'Metacritic', 'Value': '44/100'}],
               'RottenRating': '38',
               'Runtime': '108 min',
               'Title': 'The Marksman',
               'Writer': 'Chris Charles, Danny Kravitz, Robert Lorenz',
               'imdbRating': '5.6',
               'imdbVotes': '21,021'},
 'tt6911608': {'Actors': 'Lily James, Amanda Seyfried, Meryl Streep',
               'Director': 'Ol Parker',
               'Genre': 'Comedy, Musical, Romance',
               'MetacriticRating': '60',
               'Plot': 'Five years after the events of Mamma Mia! (2008), '
                       'Sophie prepares for the grand reopening of the Hotel '
                       "Bella Donna as she learns more about her mother's "
                       'past.',
               'Rated': 'PG-13',
               'Ratings': [{'Source': 'Intern

               'Title': "A Dog's Journey",
               'Writer': 'W. Bruce Cameron, Cathryn Michon, Maya Forbes',
               'imdbRating': '7.5',
               'imdbVotes': '21,765'},
 'tt8544498': {'Actors': 'Ben Affleck, Al Madrigal, Janina Gavankar',
               'Director': "Gavin O'Connor",
               'Genre': 'Drama, Sport',
               'MetacriticRating': '66',
               'Plot': 'Jack Cunningham was a high school basketball phenom '
                       'who walked away from the game, forfeiting his future. '
                       'Years later, when he reluctantly accepts a coaching '
                       'job at his alma mater, he may get one last shot at '
                       'redemption.',
               'Rated': 'R',
               'Ratings': [{'Source': 'Internet Movie Database',
                            'Value': '6.7/10'},
                           {'Source': 'Rotten Tomatoes', 'Value': '84%'},
                           {'Source': 'Metacr

In [81]:


omdb_att_df = pd.DataFrame(omdb_attrs).transpose()
omdb_att_df.columns = ['Actors', 'Director', 'Genre', 'Plot', 'Rated', 'Ratings', 'Runtime', 'Writer', 'Title', 'imdbRating', 'imdbVotes', 'RottenRating', 'MetacriticRating']
omdb_att_df


,Actors,Director,Genre,Plot,Rated,Ratings,Runtime,Writer,Title,imdbRating,imdbVotes,RottenRating,MetacriticRating
tt5433138,"Vin Diesel, Michelle Rodriguez, Jordana Brewster",Justin Lin,"Action, Adventure, Crime",Dom and the crew must take on an international...,PG-13,"[{'Source': 'Internet Movie Database', 'Value'...",143 min,"Daniel Casey, Justin Lin, Alfredo Botello",F9,5.2,"85,231",59,58
tt5034838,"Alexander Skarsgård, Millie Bobby Brown, Rebec...",Adam Wingard,"Action, Sci-Fi, Thriller",The epic next chapter in the cinematic Monster...,PG-13,"[{'Source': 'Internet Movie Database', 'Value'...",113 min,"Terry Rossio, Michael Dougherty, Zach Shields",Godzilla vs. Kong,6.4,"166,316",75,59
tt3480822,"Scarlett Johansson, Florence Pugh, David Harbour",Cate Shortland,"Action, Adventure, Sci-Fi",Natasha Romanoff confronts the darker parts of...,PG-13,"[{'Source': 'Internet Movie Database', 'Value'...",134 min,"Eric Pearson, Jac Schaeffer, Ned Benson",Black Widow,6.8,"187,336",79,67
tt8332922,"Emily Blunt, Millicent Simmonds, Cillian Murphy",John Krasinski,"Drama, Horror, Sci-Fi","Following the events at home, the Abbott famil...",PG-13,"[{'Source': 'Internet Movie Database', 'Value'...",97 min,"John Krasinski, Bryan Woods, Scott Beck",A Quiet Place Part II,7.3,"150,990",91,71
tt6264654,"Ryan Reynolds, Jodie Comer, Taika Waititi",Shawn Levy,"Action, Comedy, Sci-Fi",A bank teller discovers that he's actually an ...,PG-13,"[{'Source': 'Internet Movie Database', 'Value'...",115 min,"Matt Lieberman, Zak Penn",Free Guy,7.6,"47,740",NaN,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt7371706,"David LeRoy Anderson, Drew Goddard, Heather La...",Drew Goddard,"Documentary, Short",A documentary that explores how the monsters w...,N/A,"[{'Source': 'Internet Movie Database', 'Value'...",12 min,N/A,The Cabin In The Woods: An Army of Nightmares ...,6.8,45,NaN,NaN
tt1781769,"Keira Knightley, Jude Law, Aaron Taylor-Johnson",Joe Wright,"Drama, Romance","In late-19th-century Russian high society, St....",R,"[{'Source': 'Internet Movie Database', 'Value'...",129 min,"Tom Stoppard, Lev Tolstoy",Anna Karenina,6.6,"95,779",63,63
tt1298649,"Ben Stiller, Vince Vaughn, Jonah Hill",Akiva Schaffer,"Comedy, Sci-Fi",Four men who form a neighborhood watch group a...,R,"[{'Source': 'Internet Movie Database', 'Value'...",102 min,"Jared Stern, Seth Rogen, Evan Goldberg",The Watch,5.7,"124,281",16,36
tt1748122,"Jared Gilman, Kara Hayward, Bruce Willis",Wes Anderson,"Comedy, Drama, Romance",A pair of young lovers flee their New England ...,PG-13,"[{'Source': 'Internet Movie Database', 'Value'...",94 min,"Wes Anderson, Roman Coppola",Moonrise Kingdom,7.8,"330,779",93,84


In [82]:
omdb_att_df = omdb_att_df[['Title', 'Actors', 'Director', 'Genre', 'Plot', 'Rated', 'Runtime', 'Writer', 'RottenRating', 'MetacriticRating', 'imdbRating', 'imdbVotes']]
omdb_att_df

,Title,Actors,Director,Genre,Plot,Rated,Runtime,Writer,RottenRating,MetacriticRating,imdbRating,imdbVotes
tt5433138,F9,"Vin Diesel, Michelle Rodriguez, Jordana Brewster",Justin Lin,"Action, Adventure, Crime",Dom and the crew must take on an international...,PG-13,143 min,"Daniel Casey, Justin Lin, Alfredo Botello",59,58,5.2,"85,231"
tt5034838,Godzilla vs. Kong,"Alexander Skarsgård, Millie Bobby Brown, Rebec...",Adam Wingard,"Action, Sci-Fi, Thriller",The epic next chapter in the cinematic Monster...,PG-13,113 min,"Terry Rossio, Michael Dougherty, Zach Shields",75,59,6.4,"166,316"
tt3480822,Black Widow,"Scarlett Johansson, Florence Pugh, David Harbour",Cate Shortland,"Action, Adventure, Sci-Fi",Natasha Romanoff confronts the darker parts of...,PG-13,134 min,"Eric Pearson, Jac Schaeffer, Ned Benson",79,67,6.8,"187,336"
tt8332922,A Quiet Place Part II,"Emily Blunt, Millicent Simmonds, Cillian Murphy",John Krasinski,"Drama, Horror, Sci-Fi","Following the events at home, the Abbott famil...",PG-13,97 min,"John Krasinski, Bryan Woods, Scott Beck",91,71,7.3,"150,990"
tt6264654,Free Guy,"Ryan Reynolds, Jodie Comer, Taika Waititi",Shawn Levy,"Action, Comedy, Sci-Fi",A bank teller discovers that he's actually an ...,PG-13,115 min,"Matt Lieberman, Zak Penn",NaN,62,7.6,"47,740"
...,...,...,...,...,...,...,...,...,...,...,...,...
tt7371706,The Cabin In The Woods: An Army of Nightmares ...,"David LeRoy Anderson, Drew Goddard, Heather La...",Drew Goddard,"Documentary, Short",A documentary that explores how the monsters w...,N/A,12 min,N/A,NaN,NaN,6.8,45
tt1781769,Anna Karenina,"Keira Knightley, Jude Law, Aaron Taylor-Johnson",Joe Wright,"Drama, Romance","In late-19th-century Russian high society, St....",R,129 min,"Tom Stoppard, Lev Tolstoy",63,63,6.6,"95,779"
tt1298649,The Watch,"Ben Stiller, Vince Vaughn, Jonah Hill",Akiva Schaffer,"Comedy, Sci-Fi",Four men who form a neighborhood watch group a...,R,102 min,"Jared Stern, Seth Rogen, Evan Goldberg",16,36,5.7,"124,281"
tt1748122,Moonrise Kingdom,"Jared Gilman, Kara Hayward, Bruce Willis",Wes Anderson,"Comedy, Drama, Romance",A pair of young lovers flee their New England ...,PG-13,94 min,"Wes Anderson, Roman Coppola",93,84,7.8,"330,779"


In [83]:
#omdb attributes linked by imdb code to box office db
omdb_att_df.to_csv('omdb_att_df.csv')